In [ ]:
import mysql.connector
import time

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="<password>",
    database="FINAL_URP"
)
cursor = conn.cursor()
print("=== Indexing Demonstration ===")

try:
    # Create table if not exists
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS LargeTable (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(50),
        score INT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """)
    conn.commit()

    # Insert data only if table is empty
    cursor.execute("SELECT COUNT(*) FROM LargeTable;")
    if cursor.fetchone()[0] < 5000:
        print("Inserting 5000 rows...")
        for i in range(5000):
            cursor.execute(
                "INSERT INTO LargeTable (name, score) VALUES (%s, %s);",
                (f"Name{i}", i % 100)
            )
        conn.commit()
        print("Data inserted.\n")

    # Query before indexing
    start = time.time()
    cursor.execute("SELECT * FROM LargeTable WHERE score = 50;")
    rows = cursor.fetchall()
    print(
        f"Query without index returned {len(rows)} rows in {time.time() - start:.5f} seconds")

    # EXPLAIN before index
    cursor.execute("EXPLAIN SELECT * FROM LargeTable WHERE score = 50;")
    print("Plan before index:", cursor.fetchall())

    # -------------------------------
    # Create index (correct MySQL syntax)
    # -------------------------------
    print("\nCreating index idx_score on LargeTable(score)...")

    try:
        cursor.execute("CREATE INDEX idx_score ON LargeTable(score);")
        conn.commit()
        print("Index created successfully!")
    except mysql.connector.Error as err:
        if "Duplicate key name" in str(err):
            print("Index already exists — skipping.")
        else:
            raise

    # Query after indexing
    start = time.time()
    cursor.execute("SELECT * FROM LargeTable WHERE score = 50;")
    rows = cursor.fetchall()
    print(
        f"\nQuery with index returned {len(rows)} rows in {time.time() - start:.5f} seconds")

    # EXPLAIN after index
    cursor.execute("EXPLAIN SELECT * FROM LargeTable WHERE score = 50;")
    print("Plan after index:", cursor.fetchall())

except mysql.connector.Error as err:
    print("Indexing error:", err)
